In [1]:
import json
from matplotlib import pyplot as plt
from copy import deepcopy

In [2]:
# dict of subjects (keys) and the list of publications they appear in (values).
types = json.load(open('../../../data/json/dim/all/relevant_types.json'))
type_clusters = json.load(open('../../oai_dc/types/clustered_types.json'))
depositonce = json.load(open('../../../data/json/dim/depositonce/relevant_subjects_reversed.json'))
edoc = json.load(open('../../../data/json/dim/edoc/relevant_subjects_reversed.json'))
refubium = json.load(open('../../../data/json/dim/refubium/relevant_subjects_reversed.json'))
all = {(s['subject'], s['type']): deepcopy(s['values']) for s in depositonce}
for repo in (edoc, refubium):
    for s in repo:
        if (s['subject'], s['type']) in all:
            all[(s['subject'], s['type'])] += s['values']
        else:
            all[(s['subject'], s['type'])] = s['values']

In [5]:
ddc = {k[0]: v for k, v in all.items() if k[1] == 'ddc'}

In [8]:
def is_ddc(n):
  """ True if the given number (string) complies with the DDC format. """
  return '.' in n and len(n.split('.')[0]) == 3 or \
      '.' not in n and len(n) == 3

In [11]:
classes, subclasses, more_specific, undefined = {}, {}, {}, {}
for term, values in ddc.items():
    if term[-2:] == '00':
        classes[term] = values
    elif term[-1:] == '0':
        subclasses[term] = values
    elif is_ddc(term):
        more_specific[term] = values
    else:
        undefined[term] = values
print(f'Out of the {len(ddc)} distinct DDC terms:')
print(f'\t{len(classes)} are classes, i.e the number ends in 00.')
print(f'\t{len(subclasses)} are subclasses, i.e the number ends in 0.')
print(f'\t{len(more_specific)} are more specific than that.')
print(f'\t{len(undefined)} are not DDC numbers.')

Out of the 390 distinct DDC terms:
	10 are classes, i.e the number ends in 00.
	71 are subclasses, i.e the number ends in 0.
	302 are more specific than that.
	7 are not DDC numbers.
